In [ ]:
import os
from torch_geometric.data.lightning import LightningNodeData
from utils.utils import load_data, load_ood_data
from utils.yaml import read_yaml_file
import plotly.figure_factory as ff
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.manifold import TSNE
import torch
from data.data_utils import DatasetInfo
import pandas as pd
import math
from plotly.subplots import make_subplots
from utils.plot import plot_abundance, plot_vacuity
import plotly.express as px
from GKDE_ood import DBU_OOD
from gpn_ood import GPN_OOD
from scipy import io as scio
from sklearn.metrics import roc_curve, precision_recall_curve, auc

In [ ]:
root = 'saved_models'
model_names = ['Ground truth', 'TLRSR', 'RGAE',
               'GKDE', 'GKDE-UR', 'GKDE-UR-TV',
               'GPN', 'GPN-UR', 'GPN-UR-TV']


In [ ]:
# paviau - 8
dataset = 'paviaU'
ood = 8
sigma = 0.5
vacuity_list = []
TLRSR_file = 'anomaly_score_0.200_0.010'
RGAE_file = 'anomaly_score_0.1000_100_120'
GKDE_files = ['runs-ood-GCNExp/GCNExp_1_0_0-111-v1.ckpt', 'runs-ood-GCNExp-UR/GCNExp_1_0.01_0-111-v1.ckpt','runs-ood-GCNExp-URTV/GCNExp_1_0.0001_1e-05-0-v1.ckpt']
GPN_files =  ['runs-ood-GPN/GPN_1_0_0-222.ckpt', 'runs-ood-GPN-UR/GPN_1_0.001_0-333.ckpt', 'runs-ood-GPN-URTV/GPN_1_0.0001_1e-05-0.ckpt']

In [ ]:
# # KSC - 7
# dataset = 'KSC'
# ood = 7
# sigma = 0.5
# vacuity_list = []
# TLRSR_file = 'anomaly_score_0.001_0.010'
# RGAE_file = 'anomaly_score_0.1000_150_40'
# GKDE_files = ['runs-ood-GCNExp/GCNExp_1_0_0-333.ckpt', 'runs-ood-GCNExp-UR/GCNExp_1_0.001_0-111-v1.ckpt','runs-ood-GCNExp-URTV/GCNExp_1_0.001_1e-05-0.ckpt']
# GPN_files =  ['runs-ood-GPN/GPN_1_0_0-222-v1.ckpt', 'runs-ood-GPN-UR/GPN_1_1_0-222-v1.ckpt', 'runs-ood-GPN-URTV/GPN_1_1_1e-05-333-v1.ckpt']

In [ ]:
# # KSC - 6
# dataset = 'KSC'
# ood = 6
# sigma = 0.5
# vacuity_list = []
# TLRSR_file = 'anomaly_score_0.010_0.001'
# RGAE_file = 'anomaly_score_0.1000_150_160'
# GKDE_files = ['runs-ood-GCNExp/GCNExp_1_0_0-111-v1.ckpt', 'runs-ood-GCNExp-UR/GCNExp_1_0.001_0-333-v1.ckpt','runs-ood-GCNExp-URTV/GCNExp_1_0.01_0.001-333-v1.ckpt']
# GPN_files =  ['runs-ood-GPN/GPN_1_0_0-222.ckpt', 'runs-ood-GPN-UR/GPN_1_1_0-333-v1.ckpt', 'runs-ood-GPN-URTV/GPN_1_1_0.001-333.ckpt']

In [ ]:
# # Houston - 0
# dataset = 'Houston'
# ood = 0
# sigma = 0.2
# vacuity_list = []
# TLRSR_file = 'anomaly_score_0.200_0.001'
# RGAE_file = 'anomaly_score_0.1000_150_80'
# GKDE_files = ['runs-ood-GCNExp/GCNExp_1_0_0-333-v1.ckpt', 'runs-ood-GCNExp-UR/GCNExp_1_0.0001_0-333-v1.ckpt','runs-ood-GCNExp-URTV/GCNExp_1_0.001_0.0001-444.ckpt']
# GPN_files =  ['runs-ood-GPN/GPN_1_0_0-444.ckpt', 'runs-ood-GPN-UR/GPN_1_0.001_0-222-v1.ckpt', 'runs-ood-GPN-URTV/GPN_1_1e-05_1e-05-444.ckpt']

In [ ]:
dataset = 'paviaU'
ood = 8

## load data

In [ ]:
datasetinfo = DatasetInfo(dataset)
# load config
config = read_yaml_file(path='', directory='configs', file_name=f'ood_config_{dataset.lower()}')
config['data']['ood_left_out_classes'] = [ood,]
data, num_classes = load_ood_data(config['data'])
datamodule = LightningNodeData(data, loader='full', batch_size = 1)

In [ ]:
## load alpha teacher
sigma = 0.5
data.alpha_prior = torch.load(f'teacher/alpha_prior_tuning/alpha_teacher_{config["data"]["dataset"]}_{config["data"]["ood_left_out_classes"][0]}_{sigma}.pt')

## load ground truth OOD detection result

In [ ]:
# plot ground truth vacuity map
vacuity_scores_gd = np.zeros_like(data.labeled_indices)
vacuity_scores_gd[data.ood_mask] = 1  
vacuity_2D = np.zeros((datasetinfo.m, datasetinfo.n))
vacuity_1D = vacuity_2D.reshape(-1,)
vacuity_1D[data.labeled_indices] = vacuity_scores_gd
# np.count_nonzero(vacuity_1D)
vacuity_list.append(vacuity_1D)

## load anomaly detection result

In [ ]:
# TLRSR
anomaly_score = scio.loadmat(os.path.join('anomaly_detection', 'code-for-PCA-TLRSR', f'{dataset}_result', f'{TLRSR_file}.mat'))['anomaly_score']
anomaly_score_1D = np.float32(anomaly_score.reshape(-1,))
vacuity_2D = np.zeros((datasetinfo.m, datasetinfo.n))
vacuity_1D = vacuity_2D.reshape(-1,)
vacuity_1D[data.labeled_indices] = anomaly_score_1D[data.labeled_indices]
vacuity_list.append(vacuity_1D)
# RGAE 
anomaly_score = scio.loadmat(os.path.join('anomaly_detection', 'Hyperspectral-anomaly-detection-with-RGAE-main', f'{dataset}_result', f'{RGAE_file}.mat'))['y']
anomaly_score_1D = np.float32(anomaly_score.reshape(-1,))
anomaly_score_1D = anomaly_score_1D/np.max(anomaly_score_1D)
vacuity_2D = np.zeros((datasetinfo.m, datasetinfo.n))
vacuity_1D = vacuity_2D.reshape(-1,)
vacuity_1D[data.labeled_indices] = anomaly_score_1D[data.labeled_indices]
vacuity_list.append(vacuity_1D)

## load saved model with predicted alpha vector

In [ ]:
def to_1D_vacuity(alpha, data, num_classes, dataset):
    datasetinfo = DatasetInfo(dataset)
    m, n = datasetinfo.m , datasetinfo.n
    alpha_0_dim = alpha.sum(axis=-1,keepdims=True)
    vacuity_score = num_classes/alpha_0_dim
    vacuity = np.zeros((m,n)).reshape(-1,)
    vacuity[data.labeled_indices] = vacuity_score.flatten()
    return vacuity

In [ ]:
config['model']['model_name'] = 'GCNExp'
model = DBU_OOD(config=config, data= data, num_classes = num_classes, device='cuda')

for file in GKDE_files:
    save_folder = os.path.join(config['data']['dataset'], config['data']['dataset']+ '_' + str(config['data']['ood_left_out_classes']))

    # params_version = f'-{config["model"]["seed"]}'
    # save_name = f'{config["model"]["model_name"]}_{config["model"]["uce_loss_weight"]}_{config["model"]["reconstruction_reg_weight"]}_{config["model"]["tv_vacuity_reg_weight"]}'
    # filename = save_name+'-'+params_version

    model = DBU_OOD.load_from_checkpoint(os.path.join(root, save_folder, file), 
                                                            config=config, 
                                                            data= data, 
                                                            num_classes = num_classes, 
                                                            device='cuda')
    # disable randomness, dropout, etc...
    model.eval()
    # predict with the model
    alpha = model.gnn(data).detach().cpu()
    vacuity = to_1D_vacuity(alpha, data.cpu(), num_classes, dataset)
    vacuity_list.append(vacuity)

In [ ]:
config['model']['model_name'] = 'GPN'
model = GPN_OOD(config=config, data= data, num_classes = num_classes, device='cuda')

for file in GPN_files:
    save_folder = os.path.join(config['data']['dataset'], config['data']['dataset']+ '_' + str(config['data']['ood_left_out_classes']))

    # params_version = f'-{config["model"]["seed"]}'
    # save_name = f'{config["model"]["model_name"]}_{config["model"]["uce_loss_weight"]}_{config["model"]["reconstruction_reg_weight"]}_{config["model"]["tv_vacuity_reg_weight"]}'
    # filename = save_name+'-'+params_version

    model = GPN_OOD.load_from_checkpoint(os.path.join(root, save_folder, file), 
                                                            config=config, 
                                                            data= data, 
                                                            num_classes = num_classes, 
                                                            device='cuda')
    # disable randomness, dropout, etc...
    model.eval()
    # predict with the model
    alpha = model.gnn(data).detach().cpu()
    vacuity = to_1D_vacuity(alpha, data.cpu(), num_classes, dataset)
    vacuity_list.append(vacuity)

## plot the vacuity map

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Create subplots
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(10, 20))

# Plot heatmaps for each model
for i in range(9):
    vacuity_1D = vacuity_list[i]
    vacuity = vacuity_1D.reshape(datasetinfo.m, datasetinfo.n)
    # Determine the row and column for the subplot
    row = i // 3
    col = i % 3
    
    # Plot heatmap
    im = axes[row, col].imshow(vacuity, cmap='viridis')
    
    # Set title
    axes[row, col].set_title(model_names[i], fontsize=15)
    
    # Add colorbar
    fig.colorbar(im, ax=axes[row, col], orientation='vertical')

# Adjust layout
plt.tight_layout()
# fig.subplots_adjust(hspace=0.1)
plt.savefig(f'./plots/vacuity_{dataset}_{ood}.pdf')
plt.show()


## plot the ROC and PR comparision

In [ ]:
# plot ROC and PR comparision

# get the ground truth
id_mask = data.id_test_mask
ood_mask = data.ood_test_mask
mask = ood_mask | id_mask
corrects = ood_mask

In [ ]:
color_map = {
    0: '#1f77b4',  # Muted blue
    1: '#ff7f0e',  # Safety orange
    2: '#2ca02c',  # Cooked asparagus green
    3: '#d62728',  # Brick red
    4: '#9467bd',  # Muted purple
    5: '#8c564b',  # Chestnut brown
    6: '#e377c2',  # Raspberry yogurt pink
    7: '#7f7f7f',  # Middle gray
    8: '#bcbd22'   # Curry yellow-green
}
marker_list = [".", "v", "d", "4","*"]

In [ ]:
import matplotlib.pyplot as plt

# Create subplots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
roc_ax, pr_ax = axes

# Set subplot titles
roc_ax.set_title('ROC Curve')
pr_ax.set_title('PR Curve')

# Compute and plot ROC and PR curves for each model
for i in range(1,6):
    name = model_names[i]
    vacuity_score = vacuity_list[i][data.labeled_indices]
    
    # ROC values
    fpr, tpr, _ = roc_curve(corrects[mask], vacuity_score[mask])
    roc_ax.plot(fpr, tpr, label=f'{name} - {auc(fpr, tpr):.4f}', color=color_map[i],linewidth=3 )
                # marker = marker_list[i-1], markersize = 5)
    
    # PR values
    precision, recall, _ = precision_recall_curve(corrects[mask], vacuity_score[mask])
    pr_ax.plot(recall, precision, label=f'{name} - {auc(recall, precision):.4f}', color=color_map[i], linewidth=3)
                    # marker = marker_list[i-1], markersize = 5)

# Set labels and legends
roc_ax.set_xlabel('False Positive Rate')
roc_ax.set_ylabel('True Positive Rate')
roc_ax.legend(loc = 'center')

pr_ax.set_xlabel('Recall')
pr_ax.set_ylabel('Precision')
pr_ax.legend(loc = 'center')

# Display the plots
plt.tight_layout()
fig.subplots_adjust(wspace=0.2)
plt.savefig(f'./plots/rocpr_{dataset}_{ood}.pdf')
plt.show()

## plot the single abundance map

In [ ]:
# submodel = 'test'
# fig = plot_abundance(alpha, data.cpu(), num_classes, dataset)    
# fig.update_layout(title_text=f'Abundance map: {dataset}-{ood}-{submodel}')
# fig.show()

## plot the single vacuity map

In [ ]:
# fig = plot_vacuity(alpha, data, num_classes, dataset)
# fig.update_layout(title_text=f'Vacuity_map: {dataset}-{ood}-{submodel}')

## check the UR improvement

In [ ]:
dataset = 'paviaU'
ood = 8
# GKDE_files = ['runs-ood-GCNExp/GCNExp_1_0.01_0-222-v1.ckpt', 'runs-ood-GCNExp-UR/GCNExp_1_0_0-444-v1.ckpt']
file = 'runs-ood-GCNExp/GCNExp_1_0_0-222-v1.ckpt'
config['model']['model_name'] = 'GCNExp'
model = DBU_OOD(config=config, data= data, num_classes = num_classes, device='cuda')

save_folder = os.path.join(config['data']['dataset'], config['data']['dataset']+ '_' + str(config['data']['ood_left_out_classes']))

# params_version = f'-{config["model"]["seed"]}'
# save_name = f'{config["model"]["model_name"]}_{config["model"]["uce_loss_weight"]}_{config["model"]["reconstruction_reg_weight"]}_{config["model"]["tv_vacuity_reg_weight"]}'
# filename = save_name+'-'+params_version

model = DBU_OOD.load_from_checkpoint(os.path.join(root, save_folder, file), 
                                                        config=config, 
                                                        data= data, 
                                                        num_classes = num_classes, 
                                                        device='cuda')
# disable randomness, dropout, etc...
model.eval()
# predict with the model
alpha = model.gnn(data).detach().cpu()
submodel = 'UCE'
fig = plot_abundance(alpha, data.cpu(), num_classes, dataset)    
fig.update_layout(title_text=f'Abundance map: {dataset}-{ood}-{submodel}')
fig.show()
fig = plot_vacuity(alpha, data, num_classes, dataset)
fig.update_layout(title_text=f'Vacuity_map: {dataset}-{ood}-{submodel}')
fig.show()


In [ ]:
dataset = 'paviaU'
ood = 8
# GKDE_files = ['runs-ood-GCNExp/GCNExp_1_0.01_0-222-v1.ckpt', 'runs-ood-GCNExp-UR/GCNExp_1_0_0-444-v1.ckpt']
file = 'runs-ood-GCNExp-TV/GCNExp_1_0_1e-05-222-v1.ckpt'
config['model']['model_name'] = 'GCNExp'
model = DBU_OOD(config=config, data= data, num_classes = num_classes, device='cuda')

save_folder = os.path.join(config['data']['dataset'], config['data']['dataset']+ '_' + str(config['data']['ood_left_out_classes']))

# params_version = f'-{config["model"]["seed"]}'
# save_name = f'{config["model"]["model_name"]}_{config["model"]["uce_loss_weight"]}_{config["model"]["reconstruction_reg_weight"]}_{config["model"]["tv_vacuity_reg_weight"]}'
# filename = save_name+'-'+params_version

model = DBU_OOD.load_from_checkpoint(os.path.join(root, save_folder, file), 
                                                        config=config, 
                                                        data= data, 
                                                        num_classes = num_classes, 
                                                        device='cuda')
# disable randomness, dropout, etc...
model.eval()
# predict with the model
alpha = model.gnn(data).detach().cpu()
submodel = 'TV'
fig = plot_abundance(alpha, data.cpu(), num_classes, dataset)    
fig.update_layout(title_text=f'Abundance map: {dataset}-{ood}-{submodel}')
fig.show()
fig = plot_vacuity(alpha, data, num_classes, dataset)
fig.update_layout(title_text=f'Vacuity_map: {dataset}-{ood}-{submodel}')
fig.show()

## plot the ROC/PR plots for the alpha teacher

In [ ]:
vacuity_list = []
# plot ground truth vacuity map
vacuity_scores_gd = np.zeros_like(data.labeled_indices)
vacuity_scores_gd[data.ood_mask] = 1  
vacuity_2D = np.zeros((datasetinfo.m, datasetinfo.n))
vacuity_1D = vacuity_2D.reshape(-1,)
vacuity_1D[data.labeled_indices] = vacuity_scores_gd
# np.count_nonzero(vacuity_1D)
vacuity_list.append(vacuity_1D)
# plot alpha prior
sigma_list = [0.1, 0.2, 0.5,1,2,5,10]
for sigma in sigma_list:
    alpha_prior = torch.load(f'teacher/alpha_prior_tuning/alpha_teacher_{config["data"]["dataset"]}_{config["data"]["ood_left_out_classes"][0]}_{sigma}.pt')
    vacuity = to_1D_vacuity(alpha_prior, data, num_classes, dataset)
    vacuity_list.append(vacuity)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

model_names = {0: 'Ground truth'}
model_names.update({i+1: f'sigma = {sigma_list[i]}' for i in range(len(sigma_list))})
# Create subplots
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(15, 10))

# Plot heatmaps for each model
for i in range(8):
    vacuity_1D = vacuity_list[i]
    vacuity = vacuity_1D.reshape(datasetinfo.m, datasetinfo.n)
    # Determine the row and column for the subplot
    row = i // 4
    col = i % 4
    
    # Plot heatmap
    im = axes[row, col].imshow(vacuity, cmap='viridis')
    
    # Set title
    axes[row, col].set_title(model_names[i], fontsize=15)
    
    # Add colorbar
    fig.colorbar(im, ax=axes[row, col], orientation='vertical')

# Adjust layout
plt.tight_layout()
# fig.subplots_adjust(hspace=0.1)
plt.savefig(f'./plots/vacuity_{dataset}_{ood}_alpha_prior.pdf')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Create subplots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
roc_ax, pr_ax = axes

# Set subplot titles
roc_ax.set_title('ROC Curve')
pr_ax.set_title('PR Curve')

# Compute and plot ROC and PR curves for each model
for i in range(1,8):
    name = model_names[i]
    vacuity_score = vacuity_list[i][data.labeled_indices]
    
    # ROC values
    fpr, tpr, _ = roc_curve(corrects[mask], vacuity_score[mask])
    roc_ax.plot(fpr, tpr, label=f'{name} - {auc(fpr, tpr):.4f}', color=color_map[i])
    
    # PR values
    precision, recall, _ = precision_recall_curve(corrects[mask], vacuity_score[mask])
    pr_ax.plot(recall, precision, label=f'{name} - {auc(recall, precision):.4f}', color=color_map[i])

# Set labels and legends
roc_ax.set_xlabel('False Positive Rate')
roc_ax.set_ylabel('True Positive Rate')
roc_ax.legend()

pr_ax.set_xlabel('Recall')
pr_ax.set_ylabel('Precision')
pr_ax.legend()

# Display the plots
plt.tight_layout()
fig.subplots_adjust(wspace=0.2)
plt.savefig(f'./plots/rocpr_{dataset}_{ood}_alpha_prior.pdf')
plt.show()